## Describtion of the Data


|Feature |Describtion
|----------|----------|
|AIRLINE | The Air Line Companies
|DATE_OF_JOURNEY |Date of taking off the plane in (MM/DD/YYYY)
|SOURCE | The City from which the plane took off
|DESTINATION |The City to which the plane landed on 
|ROUTE |The Cities where the plane passed on from the starting point to its destination
|DEP_TIME |Time of Taking off in hours 
|ARRIVAL_TIME |Time of Landing (some of the data registered with hours and some added with date)
|DURATION | The taken duration in hours and minutes
|TOTAL_STOPS |Total Numbre of stops (0 indicates that the flight had directed directly to its station with no transient ones) 
|ADDITIONAL_INFO | Some information related to the case of the flight (meal, packs, .....)
|PRICE | Total price of the flight in thousands

In [239]:
import numpy as np #1
import pandas as pd #2
import pandas_profiling as pp #3 
from math import ceil #4
import seaborn as sns #5

import plotly.graph_objects as go  #6
import plotly.express as px  #7
from plotly.subplots import make_subplots  #8

from sklearn.model_selection import train_test_split  #9 
from sklearn.metrics import r2_score  ,mean_squared_error  #10
from sklearn.impute import SimpleImputer  #11
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder , MinMaxScaler , StandardScaler, RobustScaler #12
from sklearn.linear_model import LinearRegression  #13
 
from datasist.structdata import detect_outliers #14 

import matplotlib as mpl #15
import matplotlib.pyplot as plt #16
from geopy.distance import great_circle#17
from datetime import datetime #18
from scipy import stats #19
from collections import Counter #20
import plotly.figure_factory as ff #21
from geopy.geocoders import Nominatim #22
from category_encoders import BinaryEncoder #23


pd.set_option('mode.chained_assignment',None)
sns.set(rc={'figure.figsize':(12,12)}, font_scale = 1.3)

#Reading & Ecxploting  Data

df = pd.read_excel('C:/Users/hp/Desktop/Data Science/Lec_26/Data_Train.xlsx', na_values=['N/A', 'no', '?','NaN','No info'])
df.columns  = [i.replace(' ','_') for i in df.columns]
df.columns  = [i.upper() for i in df.columns]
df.head(3)



,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,ADDITIONAL_INFO,PRICE
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,NaN,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,NaN,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,NaN,13882


In [240]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AIRLINE          10683 non-null  object
 1   DATE_OF_JOURNEY  10683 non-null  object
 2   SOURCE           10683 non-null  object
 3   DESTINATION      10683 non-null  object
 4   ROUTE            10682 non-null  object
 5   DEP_TIME         10683 non-null  object
 6   ARRIVAL_TIME     10683 non-null  object
 7   DURATION         10683 non-null  object
 8   TOTAL_STOPS      10682 non-null  object
 9   ADDITIONAL_INFO  2338 non-null   object
 10  PRICE            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [241]:
print(df.ADDITIONAL_INFO.value_counts())
print('_________________________________')
print(f'The Null Values  = {df.ADDITIONAL_INFO.isnull().sum()} which represents {(df.ADDITIONAL_INFO.isnull().mean()*100)}% are missed')


In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: ADDITIONAL_INFO, dtype: int64
_________________________________
The Null Values  = 8345 which represents 78.1147617710381% are missed


The Null Values in additional Information could not compensated, besides the huge amount of null values will lead to its drop from the df

In [242]:
df.drop('ADDITIONAL_INFO', axis = 1 , inplace =True)
df.head(3) 

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,13882


In [ ]:
for i in ['TOTAL_STOPS','AIRLINE','SOURCE','DESTINATION']:
    print(df[i].value_counts())
    print('___________________')



The Value of the '4 stops' will shift the data besdies these airlines ['Trujet', 'Vistara Premium economy' , 'Jet Airways Business'] which have very low contribution (besides they have been experimented and lowering the r-score)

In [243]:

AIRLINES_TO_BE_DROPPED = ['Trujet', 'Vistara Premium economy' , 'Jet Airways Business']
for i in AIRLINES_TO_BE_DROPPED:
    df= df.drop(df[df['AIRLINE']==i].index, axis = 0)


In [244]:
df = df.drop((df[df['TOTAL_STOPS']== '4 stops']).index, axis = 0)


In [245]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10672 entries, 0 to 10682
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AIRLINE          10672 non-null  object
 1   DATE_OF_JOURNEY  10672 non-null  object
 2   SOURCE           10672 non-null  object
 3   DESTINATION      10672 non-null  object
 4   ROUTE            10671 non-null  object
 5   DEP_TIME         10672 non-null  object
 6   ARRIVAL_TIME     10672 non-null  object
 7   DURATION         10672 non-null  object
 8   TOTAL_STOPS      10671 non-null  object
 9   PRICE            10672 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 917.1+ KB


Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Name: AIRLINE, dtype: int64

In [246]:
df.dropna(axis= 0 , inplace = True)

In [247]:
df.head()

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,13302


In [248]:
df['DATE_OF_JOURNEY']  = pd.to_datetime(df['DATE_OF_JOURNEY'], format  = '%d/%m/%Y')
df['PRICE']  = df['PRICE'] .astype(float)

df.head()

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,3897.0
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,7662.0
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,13882.0
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,6218.0
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,13302.0


In [249]:

Dur=pd.DataFrame(df.DURATION.str.replace('h', '').str.replace('m','').str.split(' ').str[0])
Dur = Dur.rename(columns = {'DURATION':'Hours'})
Dur['Mins'] =pd.DataFrame(df.DURATION.str.replace('h', '').str.replace('m','').str.split(' ').str[1])
Dur['Mins']=Dur['Mins'].fillna(0)
Dur = Dur.astype(float)
Dur['Mins']=Dur['Mins']/60
Dur['TOTAL_HOURS']=Dur['Hours']+Dur['Mins']
Dur.head(5)


,Hours,Mins,TOTAL_HOURS
0,2.0,0.833333,2.833333
1,7.0,0.416667,7.416667
2,19.0,0.000000,19.000000
3,5.0,0.416667,5.416667
4,4.0,0.750000,4.750000


In [250]:
Dur.drop(['Hours','Mins'],axis = 1 , inplace = True )
df = pd.concat([df, Dur], axis = 1)
df.head() 

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE,TOTAL_HOURS
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,3897.0,2.833333
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,7662.0,7.416667
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,13882.0,19.000000
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,6218.0,5.416667
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,13302.0,4.750000


In [251]:
Stops_to_Values  = {'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3}
df.TOTAL_STOPS=df['TOTAL_STOPS'].map(Stops_to_Values)


In [252]:
df.head()

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE,TOTAL_HOURS
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,3897.0,2.833333
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2,7662.0,7.416667
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2,13882.0,19.000000
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1,6218.0,5.416667
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1,13302.0,4.750000


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10671 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   AIRLINE          10671 non-null  object        
 1   DATE_OF_JOURNEY  10671 non-null  datetime64[ns]
 2   SOURCE           10671 non-null  object        
 3   DESTINATION      10671 non-null  object        
 4   ROUTE            10671 non-null  object        
 5   DEP_TIME         10671 non-null  object        
 6   ARRIVAL_TIME     10671 non-null  object        
 7   DURATION         10671 non-null  object        
 8   TOTAL_STOPS      10671 non-null  int64         
 9   PRICE            10671 non-null  float64       
 10  TOTAL_HOURS      10671 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 1000.4+ KB


The Route would be converted into START_TO_END that would be abbreviated into the Source-Destination Only, Decreasing the data into only 5 unique Values (without taking into consideration the points of route itself as it would be balanced with the number of stops)

In [254]:
df['START_TO_END'] = (df.ROUTE.str.split('→').str[0]+df.ROUTE.str.split('→').str[-1]).str.join('').str.replace('  ','→')
df['START_TO_END'].value_counts()

DEL→COK    4534
CCU→BLR    2871
BLR→DEL    2190
BOM→HYD     696
MAA→CCU     380
Name: START_TO_END, dtype: int64

In [255]:

px.imshow(df.corr(), width =600 , height = 600)


In [256]:
df.head(3)

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE,TOTAL_HOURS,START_TO_END
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,3897.0,2.833333,BLR→DEL
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2,7662.0,7.416667,CCU→BLR
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2,13882.0,19.000000,DEL→COK


In [257]:
df['MONTH'] = df.DATE_OF_JOURNEY.dt.month_name()
df.head(3)

,AIRLINE,DATE_OF_JOURNEY,SOURCE,DESTINATION,ROUTE,DEP_TIME,ARRIVAL_TIME,DURATION,TOTAL_STOPS,PRICE,TOTAL_HOURS,START_TO_END,MONTH
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,3897.0,2.833333,BLR→DEL,March
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2,7662.0,7.416667,CCU→BLR,May
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2,13882.0,19.000000,DEL→COK,June


In [258]:
Values_to_be_Dropped = ['DATE_OF_JOURNEY','SOURCE',	'DESTINATION'	,'ROUTE','DEP_TIME',	'ARRIVAL_TIME',	'DURATION']
df.drop(Values_to_be_Dropped, axis = 1 , inplace = True)

In [259]:
df.head(3)

,AIRLINE,TOTAL_STOPS,PRICE,TOTAL_HOURS,START_TO_END,MONTH
0,IndiGo,0,3897.0,2.833333,BLR→DEL,March
1,Air India,2,7662.0,7.416667,CCU→BLR,May
2,Jet Airways,2,13882.0,19.000000,DEL→COK,June


In [260]:
print(df.describe())
print('_________________________________________')
print(df.describe(include = 'O'))

        TOTAL_STOPS         PRICE   TOTAL_HOURS
count  10671.000000  10671.000000  10671.000000
mean       0.823822   9059.203542     10.721105
std        0.674532   4455.083320      8.463704
min        0.000000   1759.000000      1.250000
25%        0.000000   5276.500000      2.833333
50%        1.000000   8372.000000      8.666667
75%        1.000000  12373.000000     15.500000
max        3.000000  54826.000000     47.666667
_________________________________________
            AIRLINE START_TO_END  MONTH
count         10671        10671  10671
unique            9            5      4
top     Jet Airways      DEL→COK    May
freq           3849         4534   3465


In [261]:
Numerical_Names = list(df.select_dtypes(include=['int','float']).columns)

for i in Numerical_Names:
    fig = px.scatter(df, x=i,y='PRICE', width=800, height=400)
    fig.show()

In [262]:
df.drop(df[df['PRICE']>20000].index, axis= 0 , inplace =True)

In [263]:
Categorical_Names = list(df.select_dtypes(include=['object']).columns)
for i in Categorical_Names:
    print(df[i].value_counts())
    print('____________________')


print('The Categorical Columns',Categorical_Names)
print('The Numerical Columns',Numerical_Names)

Jet Airways                          3750
IndiGo                               2052
Air India                            1727
Multiple carriers                    1182
SpiceJet                              817
Vistara                               476
Air Asia                              319
GoAir                                 193
Multiple carriers Premium economy      13
Name: AIRLINE, dtype: int64
____________________
DEL→COK    4498
CCU→BLR    2870
BLR→DEL    2096
BOM→HYD     685
MAA→CCU     380
Name: START_TO_END, dtype: int64
____________________
May      3461
June     3414
March    2576
April    1078
Name: MONTH, dtype: int64
____________________
The Categorical Columns ['AIRLINE', 'START_TO_END', 'MONTH']
The Numerical Columns ['TOTAL_STOPS', 'PRICE', 'TOTAL_HOURS']


In [264]:
# Data Splitting into features and target
X = df.drop('PRICE', axis=1)
y = df['PRICE'].copy()

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=X['MONTH'])

In [265]:
Numerical_Names.remove('PRICE')
# Standardize numerical columns
SC = MinMaxScaler()
#scaler = StandardScaler()
#scaler = RobustScaler()
X_train[Numerical_Names] = SC.fit_transform(X_train[Numerical_Names])
X_test[Numerical_Names] = SC.transform(X_test[Numerical_Names])


# Handling Categorical columns using Binary Encoding

ENCODER = BinaryEncoder(cols=[ 'AIRLINE', 'START_TO_END', 'MONTH'])
X_train = ENCODER.fit_transform(X_train)
X_test = ENCODER.transform(X_test) 
                                                     
                                                         

In [266]:
 px.histogram(df.START_TO_END)

In [267]:
px.histogram(y_train)


In [268]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

print('Training Score: ', lr.score(X_train, y_train))
print('Testing Score: ', lr.score(X_test, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))

Training Score:  0.6143619721168299
Testing Score:  0.6072617311797064
RMSE:  2501.46031927504


In [269]:
# Predicting on Test Data
prices = (lr.predict(X_test))
prices


array([10987.76485859,  9123.47470413,  7336.77611428, ...,
        7613.59220492, 10658.19628197, 12104.92133131])

In [270]:
lr.coef_

array([ 1586.33887113, -1334.31133648,  3075.20155041,  -264.00470468,
        6749.36729243,   483.88323137,  -822.23202188,  -335.24253498,
       -1360.04678209,  2759.50055606,  1510.85415229,  2400.00246742])

In [271]:
X_train 

,AIRLINE_0,AIRLINE_1,AIRLINE_2,AIRLINE_3,TOTAL_STOPS,TOTAL_HOURS,START_TO_END_0,START_TO_END_1,START_TO_END_2,MONTH_0,MONTH_1,MONTH_2
951,0,0,0,1,0.000000,0.026104,0,0,1,0,0,1
4617,0,0,1,0,0.333333,0.566265,0,1,0,0,0,1
10328,0,0,1,1,0.000000,0.036145,0,1,1,0,1,0
8068,0,0,1,0,0.333333,0.495984,0,1,1,0,1,1
7723,0,1,0,0,0.000000,0.024096,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1117,0,1,0,1,0.000000,0.038153,0,1,1,0,1,1
886,0,0,1,0,0.333333,0.301205,0,1,0,0,0,1
471,0,1,1,0,0.333333,0.341365,0,1,0,0,0,1
9400,0,1,0,0,0.000000,0.026104,0,0,1,0,0,1
